In [137]:
import pandas as pd

# Load the JSON file into a pandas DataFrame with nested structure
df = pd.read_json('Ramps New.json', orient='index')

# Set the display options to show all columns and increase the column width
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 5000


# View the first five rows of the df DataFrame with all columns
first_five_rows = df.head()
print(first_five_rows)


In [1]:
import pandas as pd
import json

# Load the JSON data into a pandas DataFrame
with open('Ramps New.json') as f:
    data = json.load(f)
df = pd.json_normalize(data, 'features')

# Extract the first 5 scores and their coordinates
coordinate_score_df = df[['geometry.coordinates', 'properties.score_1']]

coordinate_score_df = coordinate_score_df.rename(columns={'geometry.coordinates': 'coordinates', 'properties.score_1': 'score_1'})
new_df = coordinate_score_df
# Print the result
print(coordinate_score_df)

                                    coordinates    score_1
0       [-7911843.053800002, 5213855.430799998]  82.400000
1       [-7912104.434000002, 5213764.354100004]  92.800000
2      [-7912111.2201999985, 5213780.467799999]  66.341154
3        [-7912093.786999997, 5213788.37470001]  95.975000
4       [-7911853.312899999, 5213875.931500003]  87.750000
...                                         ...        ...
26207        [-7908859.5842, 5213906.350699998]   0.000000
26208        [-7908620.2472, 5213876.224299997]   0.000000
26209              [-7908621.175, 5213877.0612]   0.000000
26210        [-7908602.0033, 5213865.763899997]   0.000000
26211        [-7910820.3533, 5209827.192599997]   0.000000

[26212 rows x 2 columns]


In [2]:
import folium
import pyproj
project = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')
coordinate_score_df['latitudes'], coordinate_score_df['longitudes'] = zip(*coordinate_score_df['coordinates'].apply(lambda coord: project.transform(coord[0], coord[1])))

m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

for _, row in coordinate_score_df.iterrows():
    title = f"Score: {row['score_1']}"
    folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)


m.save('Ramp_Scores_Map.html')

In [3]:
import geopandas as gpd
from shapely.geometry import Point

sidewalks_gdf = gpd.read_file('Sidewalks.json')
points_gdf = gpd.GeoDataFrame(
    coordinate_score_df, 
    geometry=[Point(coord) for coord in coordinate_score_df['coordinates']]
)

pd.set_option('display.max_columns', None)
joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')
avg_scores = joined_gdf.groupby('SWK_ID', as_index=False)['score_1'].mean()
sidewalks_gdf = sidewalks_gdf.set_index('SWK_ID')
avg_scores = avg_scores.set_index('SWK_ID')
sidewalks_gdf = sidewalks_gdf.merge(avg_scores, on='SWK_ID', how='left')
sidewalks_gdf = sidewalks_gdf.reset_index()
sidewalks_gdf = sidewalks_gdf[['geometry', 'score_1', 'SWK_ID']]
print(sidewalks_gdf.columns)
sidewalks_gdf.info()
sidewalks_gdf.to_file('Sidewalks_without_scores.json', driver='GeoJSON')


C:\Users\debif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\debif\AppData\Local\Temp\ipykernel_23260\2661763856.py:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:6492

  joined_gdf = gpd.sjoin(points_gdf, sidewalks_gdf, op='within')


Index(['geometry', 'score_1', 'SWK_ID'], dtype='object')
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 23520 entries, 0 to 23519
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  23347 non-null  geometry
 1   score_1   0 non-null      float64 
 2   SWK_ID    23520 non-null  object  
dtypes: float64(1), geometry(1), object(1)
memory usage: 551.4+ KB


In [17]:
import geopandas as gpd
from shapely.geometry import Point

# Convert coordinates in new_df to same CRS as sidewalks file
new_df = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df['coordinates'].str[0], new_df['coordinates'].str[1]))
new_df.crs = {'init': 'epsg:6492'}  # replace with appropriate CRS

# Load sidewalks file as geopandas dataframe
sidewalks = gpd.read_file('Sidewalks_without_scores.json')


import folium
import pandas as pd
import geopandas as gpd

# Create a map
m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Add sidewalk polygons to the map
folium.Choropleth(
    geo_data=sidewalks,
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    name='Sidewalks',
    highlight=True
).add_to(m)

# Add point markers for new_df to the map
for _, row in points_gdf.iterrows():
    title = f"Score: {row['score_1']}"
    folium.Marker(location=[row['latitudes'], row['longitudes']], popup=title).add_to(m)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('test.html')




#print(sidewalks.crs)
#print(new_df.crs)
# Perform spatial join
#joined = gpd.sjoin(new_df, sidewalks, op='within')
#print(joined.columns)
#joined.info()
# Group by SWK_ID and calculate mean score_1
#result = joined.groupby('SWK_ID')['score_1'].mean()


C:\Users\debif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [18]:
boston_map = folium.Map(location=[42.361145, -71.057083], zoom_start=13)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=sidewalks_gdf,
    name='Average Score',
    data=sidewalks_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score'
).add_to(boston_map)

# Add a layer control to the map
folium.LayerControl().add_to(boston_map)

# Save the map as an HTML file
boston_map.save('sidewalks_map.html')